In [1]:
import os
import joblib # ou pickle

# --- BLOC DE CONFIGURATION DES CHEMINS (À METTRE AU DÉBUT) ---

# Le chemin vers le dossier 'models' est UN NIVEAU AU-DESSUS (../) du dossier 'notebooks'
MODELS_DIR = '../models'

# On s'assure que ce dossier existe. S'il n'existe pas, on le crée.
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

# -----------------------------------------------------------------

In [2]:
# ====================================================================
# CELLULE 1 : Imports et Configuration de la connexion à la BDD
# ====================================================================
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib
import warnings

warnings.filterwarnings('ignore')

# Configuration de la connexion à la base de données (celle du service Docker)
# IMPORTANT: 'host.docker.internal' permet au notebook (tournant localement) 
# de voir la BDD qui est dans le conteneur Docker.
DB_USER = 'root'
DB_PASSWORD = 'root'
DB_HOST = 'localhost' # ou 'host.docker.internal' si le notebook tourne en dehors de Docker
DB_PORT = '3306'
DB_NAME = 'sicda_easytime'

connection_str = f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_str)

print("Connexion à la base de données établie avec succès !")


Connexion à la base de données établie avec succès !


In [3]:
# ====================================================================
# CELLULE 2 : Chargement et Standardisation des Données (VERSION DÉFINITIVE)
# ====================================================================
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
import warnings

warnings.filterwarnings('ignore')

try:
    # On charge les tables brutes
    df_mvt_raw = pd.read_sql("SELECT * FROM mouvement", engine)
    df_users_raw = pd.read_sql("SELECT * FROM utilisateur", engine)
    df_jour_raw = pd.read_sql("SELECT * FROM jour", engine)
    df_plage_horaire_raw = pd.read_sql("SELECT * FROM plage_horaire", engine)

    # --- ÉTAPE DE NETTOYAGE ET RENOMMAGE ---
    # On travaille sur des copies pour garder les originaux intacts
    
    # Pour Mouvement
    df_mvt = df_mvt_raw.copy()
    df_mvt.rename(columns={'ID': 'mvt_id', 'BADGE': 'badge', 'DATE_MOUV': 'date_mouv'}, inplace=True)
    df_mvt['date_mouv'] = pd.to_datetime(df_mvt['date_mouv'])
    
    # Pour Utilisateur
    df_users = df_users_raw.copy()
    df_users.rename(columns={'ID': 'user_id', 'BADGE': 'badge', 'PLANNING_FK': 'planning_fk'}, inplace=True)
    
    # Pour Jour
    df_jour = df_jour_raw.copy()
    df_jour.rename(columns={'ID': 'jour_id', 'PLANNING_HEBDO_FK': 'planning_id', 'LIBELLE': 'jour_nom', 'HORAIRE_FK': 'horaire_id'}, inplace=True)
    
    # Pour Plage Horaire
    df_plage_horaire = df_plage_horaire_raw.copy()
    df_plage_horaire.rename(columns={'ID': 'plage_id', 'SMPL_HORAIRE_FK': 'horaire_id', 'DEBUT': 'heure_debut_theorique_double', 'TOL_ENTREE': 'tolerance_entree'}, inplace=True)
    
    print("Données chargées et standardisées avec succès.")

except Exception as e:
    print(f"Erreur lors du chargement ou de la standardisation des données : {e}")

Données chargées et standardisées avec succès.


In [4]:
# ====================================================================
# CELLULE 3 : Logique de Détection des Retards Réels (VERSION DÉFINITIVE)
# ====================================================================
print("Début de l'analyse pour trouver les retards réels...")

# 1. Joindre pointages et utilisateurs (maintenant les colonnes 'badge' correspondent)
df_merged = pd.merge(df_mvt, df_users[['badge', 'planning_fk']], on='badge', how='left')
df_merged.dropna(subset=['planning_fk'], inplace=True)
df_merged['planning_fk'] = df_merged['planning_fk'].astype(int)

# 2. Extraire la date et le nom du jour de la semaine
df_merged['jour_date'] = df_merged['date_mouv'].dt.date
day_map_french = {0: 'Lundi', 1: 'Mardi', 2: 'Mercredi', 3: 'Jeudi', 4: 'Vendredi', 5: 'Samedi', 6: 'Dimanche'}
df_merged['jour_nom'] = df_merged['date_mouv'].dt.dayofweek.map(day_map_french)

# 3. Ne garder que le premier pointage de chaque employé pour chaque jour
df_arrivées = df_merged.loc[df_merged.groupby(['badge', 'jour_date'])['date_mouv'].idxmin()].copy()

# 4. Chaîne de jointures
# 4.1. Joindre avec 'jour' pour trouver l'ID de l'horaire
df_step1 = pd.merge(
    df_arrivées, 
    df_jour[['planning_id', 'jour_nom', 'horaire_id']], 
    left_on=['planning_fk', 'jour_nom'], 
    right_on=['planning_id', 'jour_nom'], 
    how='left'
)

# 4.2. Joindre avec 'plage_horaire'
df_final_join = pd.merge(
    df_step1, 
    df_plage_horaire[['horaire_id', 'heure_debut_theorique_double', 'tolerance_entree']],
    on='horaire_id',
    how='left'
)

# 5. Nettoyer
df_final_join.dropna(subset=['heure_debut_theorique_double'], inplace=True)

# 6. Calculer la durée du retard
df_final_join['heure_debut_theorique'] = pd.to_datetime(df_final_join['heure_debut_theorique_double'], unit='h').dt.time
df_final_join['heure_arrivee'] = df_final_join['date_mouv'].dt.time
df_final_join['duree_retard_minutes'] = (
    (pd.to_datetime(df_final_join['heure_arrivee'].astype(str)) - 
     pd.to_datetime(df_final_join['heure_debut_theorique'].astype(str))).dt.total_seconds() / 60
)

# 7. Appliquer la tolérance
df_final_join['duree_retard_minutes'] -= df_final_join['tolerance_entree'].fillna(0)
df_final_join['est_en_retard'] = df_final_join['duree_retard_minutes'] > 0

# 8. Créer le dataset final
retards_dataset = df_final_join[df_final_join['est_en_retard']].copy()
retards_dataset = retards_dataset[['badge', 'jour_date', 'duree_retard_minutes']]

print(f"Analyse terminée. {len(retards_dataset)} retards réels ont été identifiés.")
retards_dataset.head()

Début de l'analyse pour trouver les retards réels...
Analyse terminée. 69 retards réels ont été identifiés.


,badge,jour_date,duree_retard_minutes
0,1,2016-09-12,62.000000
2,1,2023-01-21,44.283333
4,1,2023-01-24,10.033333
5,1,2023-01-25,3.700000
7,1,2023-01-30,37.100000


In [5]:
# ====================================================================
# CELLULE 4 : Création du Dataset d'Entraînement avec Features
# ====================================================================
print("Création du dataset d'entraînement final...")

# On crée une colonne cible simulée : 'action' (1=TOLERER, 0=JUSTIFICATION REQUISE)
# Logique de simulation : On tolère plus facilement les petits retards, en début de semaine, 
# et si l'employé est habituellement ponctuel.
conditions = [
    (retards_dataset['duree_retard_minutes'] <= 10),
    (retards_dataset['duree_retard_minutes'] > 30)
]
choices = [1, 0] # Tolérer les petits, ne pas tolérer les gros
retards_dataset['action'] = np.select(conditions, choices, default=np.random.choice([0, 1], size=len(retards_dataset), p=[0.6, 0.4]))


# Création des features pour le modèle
retards_dataset['jour_date'] = pd.to_datetime(retards_dataset['jour_date'])
retards_dataset['est_debut_semaine'] = (retards_dataset['jour_date'].dt.dayofweek == 0).astype(int) # Lundi = 1, sinon 0

# Feature : Nombre de retards de cet employé dans les 30 derniers jours
# C'est une opération coûteuse, on la fait de manière optimisée
retards_dataset = retards_dataset.sort_values(by=['badge', 'jour_date'])
nb_retards_par_badge = retards_dataset.groupby('badge').cumcount()
retards_dataset['nb_retards_mois_precedent'] = nb_retards_par_badge

# Feature : Charge de travail (simulée pour l'exemple)
retards_dataset['charge_travail_equipe_jour'] = np.random.uniform(0.3, 0.9, size=len(retards_dataset)).round(2)

# Sélection des colonnes finales pour le dataset
final_dataset = retards_dataset[[
    'duree_retard_minutes',
    'nb_retards_mois_precedent',
    'est_debut_semaine',
    'charge_travail_equipe_jour',
    'action'
]].copy()

# Enregistrer le dataset pour analyse future
final_dataset.to_csv('retards_data.csv', index=False)

print(f"Dataset d'entraînement 'retards_data.csv' créé avec {len(final_dataset)} exemples.")
final_dataset.head()

Création du dataset d'entraînement final...
Dataset d'entraînement 'retards_data.csv' créé avec 69 exemples.


,duree_retard_minutes,nb_retards_mois_precedent,est_debut_semaine,charge_travail_equipe_jour,action
0,62.000000,0,1,0.83,0
2,44.283333,1,0,0.31,0
4,10.033333,2,0,0.31,0
5,3.700000,3,0,0.44,1
7,37.100000,4,1,0.84,0


In [6]:
# ====================================================================
# CELLULE 5 : Entraînement et Sauvegarde du Modèle
# ====================================================================
print("Entraînement du nouveau modèle pour les retards...")

# 1. Préparer les données
X = final_dataset.drop('action', axis=1)
y = final_dataset['action']

# 2. Diviser en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Initialiser et entraîner le modèle
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# 4. Évaluer le modèle
y_pred = model.predict(X_test)
print("\n--- Rapport de Classification ---")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

# 5. Sauvegarder le modèle entraîné
model_path = os.path.join(MODELS_DIR, 'model_retards_contextuel.pkl')
joblib.dump(model, model_path)

print(f"\nModèle entraîné et sauvegardé avec succès dans : '{model_path}'")

Entraînement du nouveau modèle pour les retards...

--- Rapport de Classification ---
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         6
           1       0.73      1.00      0.84         8

    accuracy                           0.79        14
   macro avg       0.86      0.75      0.75        14
weighted avg       0.84      0.79      0.77        14

Accuracy: 0.79

Modèle entraîné et sauvegardé avec succès dans : '../models\model_retards_contextuel.pkl'
